In [647]:
import pandas as pd
import numpy as np

### 【任务四】显卡任务


* **题目描述**

 下面给出了3090显卡的性能测评日志结果，每一条日志有如下结构：
 
 > Benchmarking #2# #4# precision type #1#
 > #1#  model average #2# time :  #3# ms
 
 其中#1#代表的是模型名称，#2#的值为train(ing)或inference，表示训练状态或推断状态，#3#表示耗时，#4#表示精度，其中包含了float, half, double三种类型，下面是一个具体的例子：
 
 >Benchmarking Inference float precision type resnet50
 >resnet50  model average inference time :  13.426570892333984 ms
 
 请把日志结果进行整理，变换成如下状态，model_i用相应模型名称填充，按照字母顺序排序，数值保留三位小数：
 
 
 | -- |Train_half|Train_float|Train_double|Inference_half|Inference_float|Inference_double| 
 |--|--|--|--|--|	--|--|
 |model_1|0.954	|	0.901|	0.357|	0.281|	0.978|1.130|  
 |model_2|0.360	|0.794|	0.011|	1.083|	1.137	|0.394|  
 |...|...	|...|	...|	...|	...	|...|  
 

* **数据下载** 链接：https://pan.baidu.com/s/1CjfdtavEywHtZeWSmCGv3A 14 提取码：4mui


* **题解如下:**

In [140]:
# 简单的按行 读取数据集

df = pd.read_csv(r'../data/benchmark.txt',skiprows= 9,sep='/t',error_bad_lines = False)
df= df.iloc[:-2,:] # 去掉多余的数据
df.columns = ['Data']
df.head()

D:\app2\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,Data
0,Benchmarking Training float precision type mna...
1,mnasnet0_5 model average train time : 28.527...
2,Benchmarking Training float precision type mna...
3,mnasnet0_75 model average train time : 34.10...
4,Benchmarking Training float precision type mna...


In [142]:
#  现在的数据是每两行为一组数据 可以分别取奇索引 和偶索引 然后分别对文本数据处理

##  处理奇数索引数据 按照一定方式 split 文本数据
df1 = df.iloc[1::2,:]
df1.columns = ['Data1']
df1_split = df1['Data1'].str.split('( model average )|( time :)|( ms)',expand=True)  

## 参考答案 ： 数据都是统一格式的，可以直接用extract 匹配正则表达式
df1.head()

,Data1
1,mnasnet0_5 model average train time : 28.527...
3,mnasnet0_75 model average train time : 34.10...
5,mnasnet1_0 model average train time : 34.313...
7,mnasnet1_3 model average train time : 35.556...
9,resnet18 model average train time : 18.66008...


In [156]:
##  取Model Type Time我们需要的数据 并处理索引 方便后面拼接

df1_split = df1_split.loc[:,[0,4,8]]
df1_split.columns = ['Model','Type','Time']
df1_split.reset_index(drop=True,inplace=True)
df1_split.head()

,Model,Type,Time
0,mnasnet0_5,train,28.527636528015137
1,mnasnet0_75,train,34.10548686981201
2,mnasnet1_0,train,34.31377410888672
3,mnasnet1_3,train,35.556888580322266
4,resnet18,train,18.660082817077637


In [151]:
# 处理偶索引需要的数据

df2 = df.iloc[0::2,:]
df2.columns = ['Data2']
df2_split = df2.Data2.str.split(' ',expand = True)
df2_split.head()

,0,1,2,3,4,5
0,Benchmarking,Training,float,precision,type,mnasnet0_5
2,Benchmarking,Training,float,precision,type,mnasnet0_75
4,Benchmarking,Training,float,precision,type,mnasnet1_0
6,Benchmarking,Training,float,precision,type,mnasnet1_3
8,Benchmarking,Training,float,precision,type,resnet18


In [152]:
## 
df2_split  = df2_split.loc[:,[2]]

df2_split.columns = ['precision']
df2_split.reset_index(drop=True,inplace=True)
df2_split.head()

,2
0,float
2,float
4,float
6,float
8,float


In [157]:
# 将我们刚刚处理的数据 横向拼接起来

df_split = pd.concat([df1_split,df2_split],
                     axis = 1)

df_split.head()

,Model,Type,Time,precision
0,mnasnet0_5,train,28.527636528015137,float
1,mnasnet0_75,train,34.10548686981201,float
2,mnasnet1_0,train,34.31377410888672,float
3,mnasnet1_3,train,35.556888580322266,float
4,resnet18,train,18.660082817077637,float


In [159]:
# df_split.Type首字母大写 拼接 df_split.precision 作为新的列

df_split['Type_precision'] = df_split.Type.str.capitalize() +'_'+df_split.precision
df_split['Time'] = np.round(df_split['Time'].astype(np.float),3)
df_split.head()

,Model,Type,Time,precision,Type_precision
0,mnasnet0_5,train,28.528,float,Train_float
1,mnasnet0_75,train,34.105,float,Train_float
2,mnasnet1_0,train,34.314,float,Train_float
3,mnasnet1_3,train,35.557,float,Train_float
4,resnet18,train,18.660,float,Train_float


In [160]:
# 最后纵表 转宽表 得到我们想要的数据

df_split.pivot(index= 'Model',
               columns='Type_precision',
               values = 'Time')

Type_precision,Inference_double,Inference_float,Inference_half,Train_double,Train_float,Train_half
Model,,,,,,
densenet121,144.111,15.637,19.772,417.207,93.357,88.976
densenet161,511.177,31.750,27.555,1290.287,136.624,144.319
densenet169,175.808,21.598,26.371,511.404,104.840,121.556
densenet201,223.960,26.169,33.394,654.365,129.334,118.940
mnasnet0_5,11.870,8.039,6.929,48.232,28.528,27.198
mnasnet0_75,18.154,6.246,6.010,70.395,34.105,28.787
mnasnet1_0,25.796,6.553,6.785,88.913,34.314,29.600
mnasnet1_3,35.082,6.871,6.457,118.835,35.557,35.175
mobilenet_v2,25.479,6.430,6.394,86.310,36.189,30.085


### 【任务五】水压站点的特征工程


* **题目描述** 

    df1和df2中分别给出了18年和19年各个站点的数据，其中列中的H0至H23分别代表当天0点至23点；df3中记录了18-19年的每日该地区的天气情况，请完成如下的任务：
    
>``` python  
import pandas as pd
import numpy as np
df1 = pd.read_csv('yali18.csv')
df2 = pd.read_csv('yali19.csv')
df3 = pd.read_csv('qx1819.csv')
```

* 通过df1和df2构造df，把时间设为索引，第一列为站点编号，第二列为对应时刻的压力大小，排列方式如下（压力数值请用正确的值替换）：

>                 站点    压力
> 2018-01-01 00:00:00       1    1.0  
> 2018-01-01 00:00:00       2    1.0  
> ...                     ...    ...  
> 2018-01-01 00:00:00      30    1.0  
> 2018-01-01 01:00:00       1    1.0  
> 2018-01-01 01:00:00       2    1.0  
> ...                     ...    ...  
> 2019-12-31 23:00:00      30    1.0  


* 在上一问构造的df基础上，构造下面的特征序列或DataFrame，并把它们逐个拼接到df的右侧

    * 当天最高温、最低温和它们的温差
    * 当天是否有沙暴、是否有雾、是否有雨、是否有雪、是否为晴天
    * 选择一种合适的方法度量雨量/下雪量的大小（构造两个序列分别表示二者大小）
    * 限制只用4列，对风向进行0-1编码（只考虑风向，不考虑大小）
    

* 对df的水压一列构造如下时序特征：

    * 当前时刻该站点水压与本月的相同整点时间该站点水压均值的差，例如当前时刻为2018-05-20 17:00:00，那么对应需要减去的值为当前月所有17:00:00时间点水压值的均值
    * 当前时刻所在周的周末该站点水压均值与工作日水压均值之差
    * 当前时刻向前7日内，该站点水压的均值、标准差、0.95分位数、下雨天数与下雪天数的总和
    * 当前时刻向前7日内，该站点同一整点时间水压的均值、标准差、0.95分位数
    * 当前时刻所在日的该站点水压最高值与最低值出现时刻的时间差


* **数据下载** 链接：https://pan.baidu.com/s/1Tqad4b7zN1HBbc-4t4xc6w 8 提取码：ijbd


* **题解如下:**

In [234]:
## df1
df1 = pd.read_csv('../data/yali18.csv')
df1.head()

,Time,MeasName,H0,H1,H2,H3,H4,H5,H6,H7,...,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23
0,2018-01-01,站点4,0.402750,0.407625,0.418125,0.425250,0.426000,0.425250,0.417375,0.426375,...,0.348750,0.359250,0.355500,0.344250,0.352125,0.356250,0.347250,0.343875,0.356625,0.418875
1,2018-01-01,站点7,0.214375,0.226750,0.232375,0.233125,0.235000,0.232750,0.230875,0.220000,...,0.187375,0.196750,0.199750,0.192250,0.186250,0.183250,0.177250,0.163375,0.165250,0.199375
2,2018-01-01,站点22,0.247000,0.248125,0.271375,0.251125,0.272125,0.256375,0.257125,0.242500,...,0.245500,0.242875,0.238375,0.230875,0.237250,0.236875,0.236500,0.236500,0.241000,0.254500
3,2018-01-01,站点21,0.284250,0.289875,0.283500,0.281250,0.288375,0.288750,0.285750,0.255750,...,0.227625,0.238125,0.238500,0.218625,0.207000,0.212625,0.209250,0.189000,0.217875,0.270000
4,2018-01-01,站点20,0.292875,0.295875,0.305250,0.298875,0.310125,0.300750,0.288375,0.262500,...,0.247500,0.241125,0.243375,0.232500,0.233625,0.224250,0.219375,0.202125,0.219375,0.286500


In [235]:
## 站名保留编号 Time转变类型

df1.Time = pd.to_datetime(df1.Time)
df1.MeasName = df1.MeasName.str.extract('(\d+)')
df1.head()

,Time,MeasName,H0,H1,H2,H3,H4,H5,H6,H7,...,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23
0,2018-01-01,4,0.402750,0.407625,0.418125,0.425250,0.426000,0.425250,0.417375,0.426375,...,0.348750,0.359250,0.355500,0.344250,0.352125,0.356250,0.347250,0.343875,0.356625,0.418875
1,2018-01-01,7,0.214375,0.226750,0.232375,0.233125,0.235000,0.232750,0.230875,0.220000,...,0.187375,0.196750,0.199750,0.192250,0.186250,0.183250,0.177250,0.163375,0.165250,0.199375
2,2018-01-01,22,0.247000,0.248125,0.271375,0.251125,0.272125,0.256375,0.257125,0.242500,...,0.245500,0.242875,0.238375,0.230875,0.237250,0.236875,0.236500,0.236500,0.241000,0.254500
3,2018-01-01,21,0.284250,0.289875,0.283500,0.281250,0.288375,0.288750,0.285750,0.255750,...,0.227625,0.238125,0.238500,0.218625,0.207000,0.212625,0.209250,0.189000,0.217875,0.270000
4,2018-01-01,20,0.292875,0.295875,0.305250,0.298875,0.310125,0.300750,0.288375,0.262500,...,0.247500,0.241125,0.243375,0.232500,0.233625,0.224250,0.219375,0.202125,0.219375,0.286500


In [236]:
# 宽表转长表

df1 = df1.melt(id_vars = ['Time','MeasName'],
               var_name='Hour',
               value_name= 'Pressure')

# 将Hour 加入Time

df1.Hour = df1.Hour.str.extract('(\d+)')
df1.Time = df1.Time + pd.to_timedelta(df1.Hour.astype(np.int),unit= 'hours')
df1.head()

,Time,MeasName,Hour,Pressure
0,2018-01-01,4,0,0.402750
1,2018-01-01,7,0,0.214375
2,2018-01-01,22,0,0.247000
3,2018-01-01,21,0,0.284250
4,2018-01-01,20,0,0.292875


In [237]:
## 处理df2

df2 = pd.read_csv('../data/yali19.csv')
df2.head()

,Time,MeasName,H0,H1,H2,H3,H4,H5,H6,H7,...,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23
0,2019-01-01,站点4,0.342000,0.429375,0.429000,0.440250,0.445875,0.444750,0.417750,0.387000,...,0.319875,0.326250,0.323625,0.322500,0.309000,0.307125,0.307125,0.307125,0.307125,0.307125
1,2019-01-01,站点7,0.215125,0.239500,0.257500,0.246250,0.275125,0.264625,0.229375,0.205375,...,0.180625,0.176500,0.181375,0.155125,0.159625,0.146125,0.144625,0.135250,0.158875,0.184750
2,2019-01-01,站点22,0.244750,0.248875,0.246625,0.247375,0.247375,0.245500,0.244000,0.239500,...,0.238000,0.236125,0.235375,0.238000,0.231250,0.232375,0.226750,0.227875,0.236125,0.242125
3,2019-01-01,站点21,0.228750,0.248250,0.258750,0.252750,0.264375,0.265875,0.237750,0.208125,...,0.200625,0.202125,0.198000,0.186750,0.185250,0.180000,0.174750,0.177375,0.192750,0.212625
4,2019-01-01,站点20,0.239125,0.260500,0.269125,0.263500,0.278125,0.279625,0.250375,0.216625,...,0.212500,0.214375,0.202000,0.199000,0.195250,0.188500,0.187750,0.186625,0.205000,0.225250


In [238]:
## 处理数据

df2.Time = pd.to_datetime(df2.Time)
df2.MeasName = df2.MeasName.str.extract('(\d+)')


# 宽表转长表

df2 = df2.melt(id_vars = ['Time','MeasName'],
               var_name='Hour',
               value_name= 'Pressure')

# 将Hour 加入Time

df2.Hour = df2.Hour.str.extract('(\d+)')
df2.Time = df2.Time + pd.to_timedelta(df2.Hour.astype(np.int),unit= 'hours')
df2.head()

,Time,MeasName,Hour,Pressure
0,2019-01-01,4,0,0.342000
1,2019-01-01,7,0,0.215125
2,2019-01-01,22,0,0.244750
3,2019-01-01,21,0,0.228750
4,2019-01-01,20,0,0.239125


In [241]:
## 拼接df1 df2 得到df


df1 = df1[['Time','MeasName','Pressure']]
df2 = df2[['Time','MeasName','Pressure']]
df = pd.concat([df1,df2])
df = df.sort_values(by=['MeasName','Time'])
df.set_index('Time',inplace=True)
df.head()

,MeasName,Pressure
Time,,
2018-01-01 00:00:00,1,0.288625
2018-01-01 01:00:00,1,0.292000
2018-01-01 02:00:00,1,0.290500
2018-01-01 03:00:00,1,0.299500
2018-01-01 04:00:00,1,0.300250


* 在上一问构造的df基础上，构造下面的特征序列或DataFrame，并把它们逐个拼接到df的右侧

    * 当天最高温、最低温和它们的温差
    * 当天是否有沙暴、是否有雾、是否有雨、是否有雪、是否为晴天
    * 选择一种合适的方法度量雨量/下雪量的大小（构造两个序列分别表示二者大小）
    * 限制只用4列，对风向进行0-1编码（只考虑风向，不考虑大小）

In [649]:
# 读取数据
# df3

df3 = pd.read_csv('../data/qx1819.csv')
df3['日期'] = pd.to_datetime(df3['日期'])
df3.head()

,日期,天气,气温,风向
0,2018-01-01,多云,1C～-4C,东南风 微风
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级
2,2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级
3,2018-01-04,阴,0C～-4C,东北风转北风 3-4级转3-4级
4,2018-01-05,阴转多云,3C～-4C,西风转北风 3-4级转3-4级


* **【最高/低温】** 当天最高温、最低温和它们的温差

In [650]:
# 取出每天的最高温 最低温 

df3['最高温'] = df3['气温'].str.split('～',expand = True).iloc[:,0].str.extract('(-*\d+)')
df3['最低温'] = df3['气温'].str.split('～',expand = True).iloc[:,1].str.extract('(-*\d+)')

df3.head()

,日期,天气,气温,风向,最高温,最低温
0,2018-01-01,多云,1C～-4C,东南风 微风,1,-4
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0
2,2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1
3,2018-01-04,阴,0C～-4C,东北风转北风 3-4级转3-4级,0,-4
4,2018-01-05,阴转多云,3C～-4C,西风转北风 3-4级转3-4级,3,-4


In [651]:
# 填充缺失值 并计算温差 
# 用平均值填充我不会 计算平均值需要先对温度的数据转为int值，不然无法计算mean ，但是转为int值得时候，空值无法转为int
# 是不是可以转为float 做完再试试

df3['最高温'] = df3['最高温'].fillna(method='ffill').astype(np.int) 
df3['最低温'] = df3['最低温'].fillna(method='ffill').astype(np.int) 

df3['温差'] = df3['最高温']- df3['最低温']
df3.head()

,日期,天气,气温,风向,最高温,最低温,温差
0,2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8
2,2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2
3,2018-01-04,阴,0C～-4C,东北风转北风 3-4级转3-4级,0,-4,4
4,2018-01-05,阴转多云,3C～-4C,西风转北风 3-4级转3-4级,3,-4,7


* **【是否雨/雪】**当天是否有沙暴、是否有雾、是否有雨、是否有雪、是否为晴天

In [652]:
df3['是否有沙暴'] = df3['天气'].str.contains('(沙)').apply(lambda x:'是' if x else '否')
df3['是否有雾'] = df3['天气'].str.contains('(雾)').apply(lambda x:'是' if x else '否')
df3['是否有雨'] = df3['天气'].str.contains('(雨)').apply(lambda x:'是' if x else '否')
df3['是否有雪'] = df3['天气'].str.contains('(雪)').apply(lambda x:'是' if x else '否')
df3['是否晴天'] = df3['天气'].str.contains('(晴)').apply(lambda x:'是' if x else '否')
df3.head(10)

D:\app2\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,日期,天气,气温,风向,最高温,最低温,温差,是否有沙暴,是否有雾,是否有雨,是否有雪,是否晴天
0,2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5,否,否,否,否,否
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8,否,否,否,否,否
2,2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2,否,否,否,是,否
3,2018-01-04,阴,0C～-4C,东北风转北风 3-4级转3-4级,0,-4,4,否,否,否,否,否
4,2018-01-05,阴转多云,3C～-4C,西风转北风 3-4级转3-4级,3,-4,7,否,否,否,否,否
5,2018-01-06,多云转小到中雪,3C～0C,东北风转北风 3-4级转3-4级,3,0,3,否,否,否,是,否
6,2018-01-07,阴转小雪,0C～-2C,东北风转北风 3-4级,0,-2,2,否,否,否,是,否
7,2018-01-08,阴转晴,2C～-4C,西风转西北风 5-6级转5-6级,2,-4,6,否,否,否,否,是
8,2018-01-09,晴,5C～-6C,西风转西北风 4-5级转4-5级,5,-6,11,否,否,否,否,是
9,2018-01-10,晴,1C～-6C,西北风转北风 3-4级,1,-6,7,否,否,否,否,是


 * **【下雨（雪）量】** 选择一种合适的方法度量雨量/下雪量的大小（构造两个序列分别表示二者大小）

In [653]:
# 查看所有下雪天气名词
df3[df3['天气'].str.contains('(雪)')]['天气'].value_counts()

D:\app2\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


阴转小雪       7
多云转小雪      2
雨夹雪转阴      2
阴转雨夹雪      2
雨夹雪转多云     2
多云转小到中雪    1
大雪转多云      1
小雪转晴       1
雨夹雪转大雪     1
扬沙转雨夹雪     1
阴转中到大雪     1
雾转雨夹雪      1
Name: 天气, dtype: int64

In [656]:
# 查看所有下雨天气名词
df3[df3['天气'].str.contains('(雨)')]['天气'].value_counts()

D:\app2\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


阴转小雨       24
阴转雷阵雨      20
多云转小雨      11
阴转阵雨        8
多云转雷阵雨      7
小雨转阴        5
小雨转多云       5
阴转中雨        5
阴转小到中雨      4
小雨          4
中雨转雷阵雨      3
多云转中雨       3
多云转小到中雨     3
中雨转多云       3
小雨转雷阵雨      3
雷阵雨转多云      2
小雨转中雨       2
雷阵雨         2
小雨转阵雨       2
雾转小雨        2
阴转雨夹雪       2
雨夹雪转阴       2
雨夹雪转多云      2
雷阵雨转阴       2
多云转中到大雨     1
中雨转小雨       1
晴转阵雨        1
阴转大雨        1
中雨转阴        1
雨转多云        1
暴雨转雷阵雨      1
扬沙转雨夹雪      1
雾转雨夹雪       1
中雨转暴雨       1
大雨转多云       1
中雨转小到中雨     1
雨转阴         1
暴雨          1
雨夹雪转大雪      1
大雨转小雨       1
晴转小雨        1
多云转阵雨       1
阴转中到大雨      1
Name: 天气, dtype: int64

In [785]:

# 将雪分为 {'无雪'：无<'雨夹雪'：微量 <'小雪' 少量 <'中雪' 中量 <'大雪' 大量}
# 将雨分为 {'无雨'：无<'雨夹雪/雷阵雨'：微量 <'小雨' 少量 <'中雨' 中量 <'大雨' 大量}

def snow_category(x):
    #print(x)
    #print("##")
    ##weather = x['天气']
    
    
    #print(x.str.contains('(雨夹雪)').values)
    if x.str.contains('(雨夹雪)').values:
        return '微量'
    if x.str.contains('(小雪)').values:
        return '少量'
    if x.str.contains('(中雪)').values:
        return '中量'
    if x.str.contains('(大雪)').values:
        return '大量'
    else:
        return '无'
    
def rain_category(x):
    #print(x)
    if x.str.contains('(雨夹雪|(阵雨))').values:
        #print('1')
        return '微量'
    if x.str.contains('(小雨)').values:
        return '少量'
    if x.str.contains('(中雨)').values:
        return '中量'
    if x.str.contains('(大雨)').values:
        return '大量'
    if x.str.contains('(暴雨)').values:
        return '大大量'
    else:
        return '无'

# 这里两个函数非常类似，所以应该是可以优化的，后期继续优化

df3_tt = df3.groupby(['日期'])['天气'].agg([('下雪量',snow_category),('下雨量',rain_category)])
df3_tt = df3_tt.reset_index()
df3_tt.head()


D:\app2\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,日期,下雪量,下雨量
0,2018-01-01,无,无
1,2018-01-02,无,无
2,2018-01-03,少量,无
3,2018-01-04,无,无
4,2018-01-05,无,无


In [660]:
# 检查一下

df3[df3['天气'].str.contains('(雨夹雪)')]

D:\app2\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,日期,天气,气温,风向,最高温,最低温,温差,是否有沙暴,是否有雾,是否有雨,是否有雪,是否晴天
15,2018-01-16,雨夹雪转阴,4C～-1C,东北风转南风 3-4级转3-4级,4,-1,5,否,否,是,是,否
49,2018-02-20,阴转雨夹雪,9C～0C,东北风 3-4级,9,0,9,否,否,是,是,否
77,2018-03-20,阴转雨夹雪,5C～1C,北风 4-5级转4-5级,5,1,4,否,否,是,是,否
93,2018-04-05,雨夹雪转多云,7C～4C,北风转西北风 3-4级,7,4,3,否,否,是,是,否
366,2019-01-03,扬沙转雨夹雪,5℃～-2℃,北风转南风 3-4级转3-4级,5,-2,7,是,否,是,是,否
367,2019-01-04,雨夹雪转阴,0℃～-1℃,东北风转北风 3-4级转3-4级,0,-1,1,否,否,是,是,否
374,2019-01-11,雨夹雪转多云,1℃～-3℃,东北风转南风 <3级,1,-3,4,否,否,是,是,否
393,2019-01-30,雨夹雪转大雪,5℃～-3℃,东北风 3-4级转3-4级,5,-3,8,否,否,是,是,否
722,2019-12-25,雾转雨夹雪,4℃～0℃,东北风转北风 3-4级,4,0,4,否,是,是,是,否


In [661]:
df3_tt.loc['2018-01-16']  # 呜呼 没有问题 

snow_category    微量
rain_category    微量
Name: 2018-01-16 00:00:00, dtype: object

In [786]:
## 没有问题拼接上去

df3['下雪量'] = df3_tt['下雪量']
df3['下雨量'] = df3_tt['下雨量']
df3.head()

,日期,天气,气温,风向,最高温,最低温,温差,是否有沙暴,是否有雾,是否有雨,是否有雪,是否晴天,风向1,风向2,下雪量,下雨量
0,2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5,否,否,否,否,否,1100,0000,无,无
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8,否,否,否,否,否,1001,0000,无,无
2,2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2,否,否,否,是,否,1001,0000,少量,无
3,2018-01-04,阴,0C～-4C,东北风转北风 3-4级转3-4级,0,-4,4,否,否,否,否,否,1001,0001,无,无
4,2018-01-05,阴转多云,3C～-4C,西风转北风 3-4级转3-4级,3,-4,7,否,否,否,否,否,0010,0001,无,无


* **【风向】** 限制只用4列，对风向进行0-1编码（只考虑风向，不考虑大小）  

> * 我在理解题意上出现了障碍，4列是什么意思？怎么编码也是个问题，先简单的编码一下[东，南，西，北]  
>       
> * 东北风 [东，南，西，北]对应 [1,0,0,1] 就是东北风，那么问题来了东北风转南风这种怎么办    
>  
> * 如果是 0000-1111这样编码对应方向的话，没有必要搞四列，一列里面进行编码是不是可以了？  

> * 结合群里讨论的编码方式，用**两列数据进行风向的编码**，东北风 对应 列1 [1,0,0,1] 列2 [0,0,0,0]，东北风转南风就是 列1 [1,0,0,1] 列2 [0,1,0,0]，nice    

In [772]:
df3.head()

,日期,天气,气温,风向,最高温,最低温,温差,是否有沙暴,是否有雾,是否有雨,是否有雪,是否晴天
0,2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5,否,否,否,否,否
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8,否,否,否,否,否
2,2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2,否,否,否,是,否
3,2018-01-04,阴,0C～-4C,东北风转北风 3-4级转3-4级,0,-4,4,否,否,否,否,否
4,2018-01-05,阴转多云,3C～-4C,西风转北风 3-4级转3-4级,3,-4,7,否,否,否,否,否


In [768]:
# 获取风向
# df3.风向.str.extractall('([东南西北]{1,2})风.([东南西北]{1,2})风|([东南西北]{1,2})风')
tt = df3['风向'].str.extract('(?P<风向1>[东南西北]*[东南西北]+风).(?P<风向2>[东南西北]*[东南西北]+风|)')
tt.head()

,风向1,风向2
0,东南风,
1,东北风,
2,东北风,
3,东北风,北风
4,西风,北风


In [759]:
# 看一下数据进行编码

print(tt['风向1'].value_counts())
print(tt['风向2'].value_counts())

东北风    199
西南风     97
北风      97
南风      80
东南风     79
西风      67
东风      64
西北风     46
Name: 风向1, dtype: int64
       225
南风     164
北风     158
东风      79
东北风     40
东南风     27
西南风     13
西风      12
西北风     11
Name: 风向2, dtype: int64


In [774]:
# 对风向进行编码
# 这里编码可不可以优化？直接用字典map可以嘛？先用暴力的方法解决

wind_dict = {'':'0000',
            '东风':'1000',
            '南风':'0100',
            '西风':'0010',
            '北风':'0001',
            '东南风':'1100',
            '东北风':'1001',
            '西南风':'0110',
            '西北风':'0011'}

tt1 = tt['风向1']
tt2 = tt['风向2']
for wind,w_code in wind_dict.items():
    tt1[tt1==wind ] = w_code
    
for wind,w_code in wind_dict.items():
    tt2[tt2==wind ] = w_code    

    

print(tt1.value_counts())
print(tt2.value_counts())

# 
tt['风向1'] = tt1
tt['风向2'] = tt2
tt.head()

1001    199
0110     97
0001     97
0100     80
1100     79
0010     67
1000     64
0011     46
Name: 风向1, dtype: int64
0000    225
0100    164
0001    158
1000     79
1001     40
1100     27
0110     13
0010     12
0011     11
Name: 风向2, dtype: int64


,风向1,风向2
0,1100,0000
1,1001,0000
2,1001,0000
3,1001,0001
4,0010,0001


In [787]:
# 拼接

df3['风向1'] = tt['风向1']
df3['风向2'] = tt['风向2']
df3.head()

,日期,天气,气温,风向,最高温,最低温,温差,是否有沙暴,是否有雾,是否有雨,是否有雪,是否晴天,风向1,风向2,下雪量,下雨量
0,2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5,否,否,否,否,否,1100,0000,无,无
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8,否,否,否,否,否,1001,0000,无,无
2,2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2,否,否,否,是,否,1001,0000,少量,无
3,2018-01-04,阴,0C～-4C,东北风转北风 3-4级转3-4级,0,-4,4,否,否,否,否,否,1001,0001,无,无
4,2018-01-05,阴转多云,3C～-4C,西风转北风 3-4级转3-4级,3,-4,7,否,否,否,否,否,0010,0001,无,无


In [789]:
# 最后再拼接到原数据上，不然下一题计算不方便

* 对df的水压一列构造如下时序特征：

    * 当前时刻该站点水压与本月的相同整点时间该站点水压均值的差，例如当前时刻为2018-05-20 17:00:00，那么对应需要减去的值为当前月所有17:00:00时间点水压值的均值
    * 当前时刻所在周的周末该站点水压均值与工作日水压均值之差
    * 当前时刻向前7日内，该站点水压的均值、标准差、0.95分位数、下雨天数与下雪天数的总和
    * 当前时刻向前7日内，该站点同一整点时间水压的均值、标准差、0.95分位数
    * 当前时刻所在日的该站点水压最高值与最低值出现时刻的时间差

In [890]:
df_pre = df.copy()
df_pre = df_pre.reset_index()
df_pre.head()

,index,Time,MeasName,Pressure
0,0,2018-01-01 00:00:00,1,0.288625
1,1,2018-01-01 01:00:00,1,0.292000
2,2,2018-01-01 02:00:00,1,0.290500
3,3,2018-01-01 03:00:00,1,0.299500
4,4,2018-01-01 04:00:00,1,0.300250


* **【水压整点差】**当前时刻该站点水压与本月的相同整点时间该站点水压均值的差，例如当前时刻为2018-05-20 17:00:00，那么对应需要减去的值为当前月所有17:00:00时间点水压值的均值

In [891]:
# 按照 年，月，小时分组，每条数据减去 平均值
df_pre['水压整点差'] = df_pre.groupby([df_pre.Time.dt.year,
                                  df_pre.Time.dt.month,
                                  df_pre.Time.dt.hour,
                                  'MeasName',])['Pressure'].apply(lambda x:x-x.mean())
df_pre.head()

,index,Time,MeasName,Pressure,水压整点差
0,0,2018-01-01 00:00:00,1,0.288625,0.014988
1,1,2018-01-01 01:00:00,1,0.292000,0.014806
2,2,2018-01-01 02:00:00,1,0.290500,0.010560
3,3,2018-01-01 03:00:00,1,0.299500,0.019427
4,4,2018-01-01 04:00:00,1,0.300250,0.019742


In [892]:
## 验证一下

test = df_pre[(df_pre.Time.dt.year== 2018) & (df_pre.Time.dt.month== 1)&(df_pre.Time.dt.hour== 1) & (df_pre.MeasName== '1') ]['Pressure']
tt = test - test.mean()
tt.iloc[0]

0.01480646431446081

* **【水压工作日差】** 当前时刻所在周的周末该站点水压均值与工作日水压均值之差

In [495]:
# 这里我感觉自己绕弯了，我是先处理得到水压工作日差，然后再与原表拼接

# 按照 周，是否工作日分组，得到平均值
df_pre_gb = df_pre.groupby([df_pre.Time.dt.year,
                df_pre.Time.dt.weekofyear,
                df_pre.Time.dt.dayofweek.isin([5,6]),
                'MeasName'])['Pressure'].mean()

df_pre_gb = df_pre_gb.rename_axis(index = ['年','周编码','是否周末','站点'])
df_pre_gb = df_pre_gb.reset_index()

df_pre_gb.head()

D:\app2\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


,年,周编码,是否周末,站点,Pressure
0,2018,1,False,1,0.247120
1,2018,1,False,10,0.244784
2,2018,1,False,11,0.233130
3,2018,1,False,12,0.226758
4,2018,1,False,13,0.235557


In [496]:
# 长表转宽表 

tt = df_pre_gb.pivot(index = ['年','站点','周编码'],
                columns= ['是否周末'],
                values = 'Pressure')

tt.head()

是否周末            False     True 
年    站点 周编码                    
2018 1  1    0.247120  0.251711
        2    0.252750  0.248945
        3    0.247359  0.238461
        4    0.250631  0.250039
        5    0.248650  0.241688

In [497]:
# 计算得到工作日水压差

tt.columns = ['工作日均值','非工作日均值']
tt['工作日水压差']= tt['非工作日均值'] - tt['工作日均值']
tt = tt.reset_index()
tt.head()

,年,站点,周编码,工作日均值,非工作日均值,工作日水压差
0,2018,1,1,0.247120,0.251711,0.004591
1,2018,1,2,0.252750,0.248945,-0.003805
2,2018,1,3,0.247359,0.238461,-0.008898
3,2018,1,4,0.250631,0.250039,-0.000592
4,2018,1,5,0.248650,0.241688,-0.006963


In [499]:
# 将计算后得结果拼接到 原表


df_pre['年'] = df_pre.Time.dt.year
df_pre['周编码'] = df_pre.Time.dt.weekofyear
df_pre.head()

D:\app2\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


,Time,MeasName,Pressure,水压整点差,年,周编码
0,2018-01-01 00:00:00,1,0.288625,0.014988,2018,1
1,2018-01-01 01:00:00,1,0.292000,0.014806,2018,1
2,2018-01-01 02:00:00,1,0.290500,0.010560,2018,1
3,2018-01-01 03:00:00,1,0.299500,0.019427,2018,1
4,2018-01-01 04:00:00,1,0.300250,0.019742,2018,1


In [500]:
# 拼接
df_pre = df_pre.merge(tt,
             left_on=['年','周编码','MeasName'],
             right_on=['年','周编码','站点'],
             how = 'left')
df_pre.head()

,Time,MeasName,Pressure,水压整点差,年,周编码,站点,工作日均值,非工作日均值,工作日水压差
0,2018-01-01 00:00:00,1,0.288625,0.014988,2018,1,1,0.247120,0.251711,0.004591
1,2018-01-01 01:00:00,1,0.292000,0.014806,2018,1,1,0.247120,0.251711,0.004591
2,2018-01-01 02:00:00,1,0.290500,0.010560,2018,1,1,0.247120,0.251711,0.004591
3,2018-01-01 03:00:00,1,0.299500,0.019427,2018,1,1,0.247120,0.251711,0.004591
4,2018-01-01 04:00:00,1,0.300250,0.019742,2018,1,1,0.247120,0.251711,0.004591
...,...,...,...,...,...,...,...,...,...,...
525595,2019-12-31 19:00:00,9,0.183000,-0.003169,2019,1,9,0.238703,0.245336,0.006633
525596,2019-12-31 20:00:00,9,0.167250,-0.016016,2019,1,9,0.238703,0.245336,0.006633
525597,2019-12-31 21:00:00,9,0.178875,0.009266,2019,1,9,0.238703,0.245336,0.006633
525598,2019-12-31 22:00:00,9,0.160875,-0.019500,2019,1,9,0.238703,0.245336,0.006633


* **【7日_均值/标准差/0.95分位数】** 当前时刻向前7日内，该站点水压的均值、标准差、0.95分位数、下雨天数与下雪天数的总和

In [799]:
# 计算均值、标准差
# 计算速度有点慢

def percentile(n):
    def quan_(x):
        return x.quantile(n)
    quan_.__name__ = "%0.2f分位数"%(n)
    return quan_


seven_s = df_pre[['Pressure','Time','MeasName']].set_index(['Time'])
seven_s = seven_s.groupby('MeasName')

seven_s = seven_s.rolling('7D').agg({'Pressure':['mean','std',percentile(0.95)]})
seven_s.fillna(method= 'backfill',inplace=True)
seven_s.head()

Pressure                    
                                  mean       std   0.95分位数
MeasName Time                                             
1        2018-01-01 00:00:00  0.288625       NaN  0.288625
         2018-01-01 01:00:00  0.290313  0.002386  0.291831
         2018-01-01 02:00:00  0.290375  0.001691  0.291850
         2018-01-01 03:00:00  0.292656  0.004767  0.298375
         2018-01-01 04:00:00  0.294175  0.005346  0.300100

In [804]:
# 将计算好的数据加到原数据中

seven_s.columns = ['7日水压均值','7日水压方差','7日0.95分位数']
seven_s = seven_s.reset_index()
seven_s= seven_s['MeasName','Time','7日水压均值','7日水压方差','7日0.95分位数']
seven_s.head()

,MeasName,Time,7日水压均值,7日水压方差,7日0.95分位数
0,1,2018-01-01 00:00:00,0.288625,NaN,0.288625
1,1,2018-01-01 01:00:00,0.290313,0.002386,0.291831
2,1,2018-01-01 02:00:00,0.290375,0.001691,0.291850
3,1,2018-01-01 03:00:00,0.292656,0.004767,0.298375
4,1,2018-01-01 04:00:00,0.294175,0.005346,0.300100


In [888]:
seven_s

,MeasName,Time,7日水压均值,7日水压方差,7日0.95分位数
0,1,2018-01-01 00:00:00,0.288625,NaN,0.288625
1,1,2018-01-01 01:00:00,0.290313,0.002386,0.291831
2,1,2018-01-01 02:00:00,0.290375,0.001691,0.291850
3,1,2018-01-01 03:00:00,0.292656,0.004767,0.298375
4,1,2018-01-01 04:00:00,0.294175,0.005346,0.300100
...,...,...,...,...,...
525595,9,2019-12-31 19:00:00,0.211201,0.059960,0.317494
525596,9,2019-12-31 20:00:00,0.211087,0.060026,0.317494
525597,9,2019-12-31 21:00:00,0.211275,0.059875,0.317494
525598,9,2019-12-31 22:00:00,0.211350,0.059802,0.317494


In [805]:
df_pre.head()

,Time,MeasName,Pressure,水压整点差,年,周编码,hour,year
0,2018-01-01 00:00:00,1,0.288625,0.014988,2018,1,0,2018
1,2018-01-01 01:00:00,1,0.292000,0.014806,2018,1,1,2018
2,2018-01-01 02:00:00,1,0.290500,0.010560,2018,1,2,2018
3,2018-01-01 03:00:00,1,0.299500,0.019427,2018,1,3,2018
4,2018-01-01 04:00:00,1,0.300250,0.019742,2018,1,4,2018


In [808]:
# 
df_pre = df_pre.merge(seven_s,
                left_on=['Time','MeasName'],
                right_on = ['Time','MeasName'],
                how = 'left')
df_pre.head()

,Time,MeasName,Pressure,水压整点差,年,周编码,hour,year,7日水压均值,7日水压方差,7日0.95分位数
0,2018-01-01 00:00:00,1,0.288625,0.014988,2018,1,0,2018,0.288625,NaN,0.288625
1,2018-01-01 01:00:00,1,0.292000,0.014806,2018,1,1,2018,0.290313,0.002386,0.291831
2,2018-01-01 02:00:00,1,0.290500,0.010560,2018,1,2,2018,0.290375,0.001691,0.291850
3,2018-01-01 03:00:00,1,0.299500,0.019427,2018,1,3,2018,0.292656,0.004767,0.298375
4,2018-01-01 04:00:00,1,0.300250,0.019742,2018,1,4,2018,0.294175,0.005346,0.300100


* **【7日整点_均值/标准差/0.95分位数】** 当前时刻向前7日内，该站点同一整点时间水压的均值、标准差、0.95分位数

In [848]:
# 和上题一样，只不过分组是年，月，整点时间+站点分组，然后滑动窗口不是7D，而是7

df_pre['hour'] = df_pre.Time.dt.hour
df_pre['day'] = df_pre.Time.dt.day
df_pre['year'] = df_pre.Time.dt.year

# 计算均值、标准差

def percentile(n):
    def quan_(x):
        return x.quantile(n)
    quan_.__name__ = "%0.2f分位数"%(n)
    return quan_


seven_hour_s = df_pre[['Pressure','year','MeasName','hour','day']].set_index(['day'])
seven_hour_s = seven_hour_s.groupby(['year','MeasName','hour'])

seven_hour_s = seven_hour_s.rolling(7).agg({'Pressure':['mean','std',percentile(0.95)]})
#seven_hour_s = seven_hour_s.rolling(7).mean()
## 
seven_hour_s.fillna(method= 'backfill',inplace = True)
seven_hour_s.head()

Pressure                    
                            mean       std   0.95分位数
year MeasName hour day                              
2018 1        0    1    0.284286  0.008529  0.291663
                   2    0.284286  0.008529  0.291663
                   3    0.284286  0.008529  0.291663
                   4    0.284286  0.008529  0.291663
                   5    0.284286  0.008529  0.291663

In [849]:
# 将计算好的数据加到原数据中

seven_hour_s.columns = ['7日整点水压均值','7日整点水压方差','7日整点0.95分位数']
seven_hour_s = seven_hour_s.reset_index()
seven_hour_s.head()

,year,MeasName,hour,day,7日整点水压均值,7日整点水压方差,7日整点0.95分位数
0,2018,1,0,1,0.284286,0.008529,0.291663
1,2018,1,0,2,0.284286,0.008529,0.291663
2,2018,1,0,3,0.284286,0.008529,0.291663
3,2018,1,0,4,0.284286,0.008529,0.291663
4,2018,1,0,5,0.284286,0.008529,0.291663


In [887]:
seven_hour_s

,year,MeasName,hour,day,7日整点水压均值,7日整点水压方差,7日整点0.95分位数
0,2018,1,0,1,0.284286,0.008529,0.291663
1,2018,1,0,2,0.284286,0.008529,0.291663
2,2018,1,0,3,0.284286,0.008529,0.291663
3,2018,1,0,4,0.284286,0.008529,0.291663
4,2018,1,0,5,0.284286,0.008529,0.291663
...,...,...,...,...,...,...,...
525595,2019,9,23,27,0.211982,0.008233,0.222450
525596,2019,9,23,28,0.210750,0.008563,0.222450
525597,2019,9,23,29,0.211768,0.009412,0.222562
525598,2019,9,23,30,0.212411,0.010254,0.224700


In [854]:
#df_pre[(df_pre.MeasName=='1')&(df_pre.hour==0)&(df_pre.year==2018)]

In [853]:
#seven_hour_s[(seven_hour_s.MeasName=='1')&(seven_hour_s.hour==0)&(seven_hour_s.year==2018)]

In [855]:

# 
df_pre = df_pre.merge(seven_hour_s,
                 left_on=['MeasName','hour','year','day'],
                 right_on = ['MeasName','hour','year','day'],
                 how = 'outer')
 

df_pre.head()

,MeasName,hour,year,Time,Pressure,水压整点差,年,周编码,7日水压均值,7日水压方差,7日0.95分位数,day,7日整点水压均值,7日整点水压方差,7日整点0.95分位数
0,1,0,2018,2018-01-01,0.288625,0.014988,2018,1,0.288625,NaN,0.288625,1,0.284286,0.008529,0.291663
1,1,0,2018,2018-01-01,0.288625,0.014988,2018,1,0.288625,NaN,0.288625,1,0.274161,0.007602,0.282925
2,1,0,2018,2018-01-01,0.288625,0.014988,2018,1,0.288625,NaN,0.288625,1,0.268321,0.009265,0.278387
3,1,0,2018,2018-01-01,0.288625,0.014988,2018,1,0.288625,NaN,0.288625,1,0.274643,0.004530,0.281613
4,1,0,2018,2018-01-01,0.288625,0.014988,2018,1,0.288625,NaN,0.288625,1,0.258464,0.014641,0.274825


* **【高低压时间差】**当前时刻所在日的该站点水压最高值与最低值出现时刻的时间差

In [576]:
#  先分组 自定义计算时间差的函数

high_low_df  = df_pre.copy()
high_low_df.head()

,Time,MeasName,Pressure,水压整点差,年,周编码
0,2018-01-01 00:00:00,1,0.288625,0.014988,2018,1
1,2018-01-01 01:00:00,1,0.292000,0.014806,2018,1
2,2018-01-01 02:00:00,1,0.290500,0.010560,2018,1
3,2018-01-01 03:00:00,1,0.299500,0.019427,2018,1
4,2018-01-01 04:00:00,1,0.300250,0.019742,2018,1


In [791]:
high_low_df['day'] = high_low_df.Time.dt.day
high_low_df['month'] = high_low_df.Time.dt.month
high_low_df['year'] = high_low_df.Time.dt.year

def my_function(x):
    pre = x['Pressure']
    time = x['Time']
    
    # 得到最高 最低水压的值
    pre_max = pre.max()
    pre_min = pre.min()
    
    
    # 根据索引得到 时间戳
    pre_max_time = time[x['Pressure']==pre_max].values
    pre_min_time = time[x['Pressure']==pre_min].values

    # 返回时间戳的差值
    
    ddelta = pd.Timedelta( pre_min_time[0]-pre_max_time[0])
    #print([ ddelta for _ in range(x.shape[0])])
    #print("#####")
    
    return ddelta

time_tt = high_low_df.groupby(['year','month','day','MeasName'])['Pressure','Time'].apply(my_function)
time_tt.head()

D:\app2\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


year  month  day  MeasName
2018  1      1    1          0 days 06:00:00
                  10         0 days 06:00:00
                  11         0 days 08:00:00
                  12         0 days 06:00:00
                  13         0 days 06:00:00
dtype: timedelta64[ns]

In [859]:
time_tt = time_tt.reset_index()
time_tt.columns = ['year', 'month', 'day', 'MeasName', '高低压时间差']
time_tt.head()

,year,month,day,MeasName,0
0,2018,1,1,1,0 days 06:00:00
1,2018,1,1,10,0 days 06:00:00
2,2018,1,1,11,0 days 08:00:00
3,2018,1,1,12,0 days 06:00:00
4,2018,1,1,13,0 days 06:00:00
...,...,...,...,...,...
21895,2019,12,31,5,0 days 16:00:00
21896,2019,12,31,6,0 days 00:00:00
21897,2019,12,31,7,0 days 19:00:00
21898,2019,12,31,8,0 days 16:00:00


In [862]:
time_tt.head()

,year,month,day,MeasName,高低压时间差
0,2018,1,1,1,0 days 06:00:00
1,2018,1,1,10,0 days 06:00:00
2,2018,1,1,11,0 days 08:00:00
3,2018,1,1,12,0 days 06:00:00
4,2018,1,1,13,0 days 06:00:00


In [877]:
# 接下来就可以把结果拼接上去了

df_pre['month'] = df_pre.Time.dt.month
df_pre = df_pre.merge(time_tt,
                 left_on=['MeasName','month','year','day'],
                 right_on = ['MeasName','month','year','day'],
                 how = 'left')
df_pre.head()

,MeasName,hour,year,Time,Pressure,水压整点差,年,周编码,7日水压均值,7日水压方差,7日0.95分位数,day,7日整点水压均值,7日整点水压方差,7日整点0.95分位数,month
0,1,0,2018,2018-01-01,0.288625,0.014988,2018,1,0.288625,0.020327,0.288625,1,0.284286,0.008529,0.291663,1
1,1,0,2018,2018-01-01,0.288625,0.014988,2018,1,0.288625,0.020327,0.288625,1,0.274161,0.007602,0.282925,1
2,1,0,2018,2018-01-01,0.288625,0.014988,2018,1,0.288625,0.020327,0.288625,1,0.268321,0.009265,0.278387,1
3,1,0,2018,2018-01-01,0.288625,0.014988,2018,1,0.288625,0.020327,0.288625,1,0.274643,0.004530,0.281613,1
4,1,0,2018,2018-01-01,0.288625,0.014988,2018,1,0.288625,0.020327,0.288625,1,0.258464,0.014641,0.274825,1


In [889]:
df_pre

,MeasName,hour,year,Time,Pressure,水压整点差,年,周编码,7日水压均值,7日水压方差,7日0.95分位数,day,7日整点水压均值,7日整点水压方差,7日整点0.95分位数,month
0,1,0,2018,2018-01-01 00:00:00,0.288625,0.014988,2018,1,0.288625,0.020327,0.288625,1,0.284286,0.008529,0.291663,1
1,1,0,2018,2018-01-01 00:00:00,0.288625,0.014988,2018,1,0.288625,0.020327,0.288625,1,0.274161,0.007602,0.282925,1
2,1,0,2018,2018-01-01 00:00:00,0.288625,0.014988,2018,1,0.288625,0.020327,0.288625,1,0.268321,0.009265,0.278387,1
3,1,0,2018,2018-01-01 00:00:00,0.288625,0.014988,2018,1,0.288625,0.020327,0.288625,1,0.274643,0.004530,0.281613,1
4,1,0,2018,2018-01-01 00:00:00,0.288625,0.014988,2018,1,0.288625,0.020327,0.288625,1,0.258464,0.014641,0.274825,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6225115,9,23,2019,2019-12-31 23:00:00,0.201375,-0.023988,2019,1,0.211321,0.059806,0.317494,31,0.273482,0.010810,0.287513,12
6225116,9,23,2019,2019-12-31 23:00:00,0.201375,-0.023988,2019,1,0.211321,0.059806,0.317494,31,0.261911,0.009746,0.273600,12
6225117,9,23,2019,2019-12-31 23:00:00,0.201375,-0.023988,2019,1,0.211321,0.059806,0.317494,31,0.262500,0.012127,0.277050,12
6225118,9,23,2019,2019-12-31 23:00:00,0.201375,-0.023988,2019,1,0.211321,0.059806,0.317494,31,0.210750,0.009156,0.222525,12


In [884]:
df3.head()

,日期,天气,气温,风向,最高温,最低温,温差,是否有沙暴,是否有雾,是否有雨,是否有雪,是否晴天,风向1,风向2,下雪量,下雨量
0,2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5,否,否,否,否,否,1100,0000,无,无
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8,否,否,否,否,否,1001,0000,无,无
2,2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2,否,否,否,是,否,1001,0000,少量,无
3,2018-01-04,阴,0C～-4C,东北风转北风 3-4级转3-4级,0,-4,4,否,否,否,否,否,1001,0001,无,无
4,2018-01-05,阴转多云,3C～-4C,西风转北风 3-4级转3-4级,3,-4,7,否,否,否,否,否,0010,0001,无,无


In [883]:
## 最后拼接df_pre和df3
## 这里df_pre 出现了很多重复值，需要检查一下，

df_pre['date'] = df_pre.Time.dt.date
df_pre.merge(df3,
            left_on ='date',
            right_on= '日期',
            how='left')

Index(['MeasName', 'hour', 'year', 'Time', 'Pressure', '水压整点差', '年', '周编码',
       '7日水压均值', '7日水压方差', '7日0.95分位数', 'day', '7日整点水压均值', '7日整点水压方差',
       '7日整点0.95分位数', 'month'],
      dtype='object')

###  想说的的话

* 第一题还行，第二题难，我花了很长的时间，学艺不精，需多多磨练。

* 感谢DateWhale组织，见识了很多厉害的同学，助教和各位老师辛苦了，组织这样的活动全靠大家的热爱和责任，非常不容易。